In [11]:
%cd /content
!git clone https://github.com/VarshiniAG/ml-intern-assessment.git
%cd "ml-intern-assessment/AIML Intern Assignment"
!ls


/content
fatal: destination path 'ml-intern-assessment' already exists and is not an empty directory.
/content/ml-intern-assessment/AIML Intern Assignment
evaluation.md  notebook  README.md  src  tests


In [12]:
import sys
import os

src_path = os.path.join(os.getcwd(), "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"Added {src_path} to sys.path")


Added /content/ml-intern-assessment/AIML Intern Assignment/src to sys.path


In [14]:
%%writefile src/attention.py
import numpy as np

def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Compute Scaled Dot-Product Attention (batch-compatible).

    Args:
        Q: (batch, seq_q, d_k)
        K: (batch, seq_k, d_k)
        V: (batch, seq_v, d_v)
        mask: (batch, seq_q, seq_k) or (seq_q, seq_k)

    Returns:
        output: (batch, seq_q, d_v)
        attention_weights: (batch, seq_q, seq_k)
    """
    # Step 1: Compute raw attention scores
    scores = np.matmul(Q, np.transpose(K, (0, 2, 1)))  # (batch, seq_q, seq_k)

    # Step 2: Scale by sqrt(d_k)
    d_k = K.shape[-1]
    scaled_scores = scores / np.sqrt(d_k)

    # Step 3: Apply mask
    if mask is not None:
        if mask.ndim == 2:
            mask = mask[None, :, :]  # broadcast to batch
        scaled_scores = np.where(mask == 0, -1e9, scaled_scores)

    # Step 4: Softmax over last axis
    exp_scores = np.exp(scaled_scores - np.max(scaled_scores, axis=-1, keepdims=True))
    attention_weights = exp_scores / np.sum(exp_scores, axis=-1, keepdims=True)

    # Step 5: Multiply weights with values
    output = np.matmul(attention_weights, V)

    return output, attention_weights



Overwriting src/attention.py


In [15]:
from src.attention import scaled_dot_product_attention
import numpy as np

# Example batch input
Q = np.random.rand(2, 3, 4).astype(np.float32)  # (batch=2, seq_q=3, d_k=4)
K = np.random.rand(2, 3, 4).astype(np.float32)
V = np.random.rand(2, 3, 5).astype(np.float32)

out, weights = scaled_dot_product_attention(Q, K, V)
print("Output shape:", out.shape)       # Expected: (2, 3, 5)
print("Weights shape:", weights.shape)  # Expected: (2, 3, 3)


Output shape: (2, 3, 5)
Weights shape: (2, 3, 3)


In [16]:
!pytest -q --disable-warnings --tb=short


....                                                                     [100%]
4 passed in 0.12s
